In [20]:
import requests
import pandas as pd
import numpy as np
import datetime
import os
import zipfile

In [28]:
#List of countries and regions are here: https://unstats.un.org/unsd/methodology/m49/overview/ cannot be downloaded
#Using a github page instead

url = "https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/raw/master/all/all.csv"
filename = "countries.csv"

response = requests.get(url)

if response.status_code == 200:
    with open(filename, "wb") as file:
        file.write(response.content)
    print("CSV file downloaded successfully.")
else:
    print("Failed to download the CSV file.")

CSV file downloaded successfully.


In [29]:
df_countries=pd.read_csv("countries.csv")

In [30]:
df_countries.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [27]:
import requests
from bs4 import BeautifulSoup

# Send a GET request to the webpage
url = "https://unstats.un.org/unsd/methodology/m49/overview/#"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find the CSV button element
    csv_button = soup.find("a", {"class": "download-link"}, text="CSV")

    if csv_button:
        csv_url = csv_button["href"]

        # Download the CSV file
        csv_response = requests.get(csv_url)

        if csv_response.status_code == 200:
            filename = "m49_countries.csv"
            with open(filename, "wb") as file:
                file.write(csv_response.content)
            print("CSV file downloaded successfully.")
        else:
            print("Failed to download the CSV file.")
    else:
        print("CSV button not found.")
else:
    print("Failed to access the webpage.")


CSV button not found.


C:\Users\varjak\AppData\Local\Temp\ipykernel_49992\1249559162.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  csv_button = soup.find("a", {"class": "download-link"}, text="CSV")


In [2]:
#To find a file of your interest: https://data.worldbank.org/indicator
#Clicking on a subset allows to get address to download csv-file

#These are addresses to obtain the data for GDP per capita, life expectancy and literacy rate
url_list=["https://api.worldbank.org/v2/en/indicator/NY.GDP.PCAP.PP.CD?downloadformat=csv", \
          "https://api.worldbank.org/v2/en/indicator/SP.DYN.LE00.IN?downloadformat=csv",\
            "https://api.worldbank.org/v2/en/indicator/SE.ADT.LITR.ZS?downloadformat=csv"]

In [3]:
#Also, giving names for the data to be obtained
data_list=["gdp_per_capita", "life_expectancy", "literacy"]

In [4]:
#First, making folder to download the data from web, names based on data_list
for d in data_list:
    if not os.path.exists(d):
        os.makedirs(d)
    if not os.path.exists(d+"_unpacked"):
        os.makedirs(d+"_unpacked")

In [5]:
#Here one by one, I dowload the data and put into a right folder
for d, u in zip(data_list, url_list):
    response = requests.get(u)
    if response.status_code == 200:
        with open(d+"/"+d+".zip", "wb") as file:
            file.write(response.content)
        print("Zip file downloaded successfully!")
    else:
        print("Failed to retrieve the zip file.")

Zip file downloaded successfully!
Zip file downloaded successfully!
Zip file downloaded successfully!


In [6]:
#Unpacking the data as well
for d in data_list:
    zip_file_path=d+"/"+d+".zip"
    destination_folder=d+"_unpacked"
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(destination_folder)
    print("Extraction completed!")


Extraction completed!
Extraction completed!
Extraction completed!


In [7]:
#To remove excess rows in the csv file, an additional method:
def copy_file(source_file, destination_file):
    with open(source_file, 'r') as source:
        with open(destination_file, 'w') as destination:
            counter=0
            for line in source:
                if(counter>3):
                    destination.write(line)
                counter=counter+1

In [8]:
#Removing extra rows from the csv file:
for d in data_list:
    file_list=os.listdir(d+"_unpacked")
    source_file=d+"_unpacked/"+file_list[0]
    destination_file=d+".csv"
    copy_file(source_file, destination_file)

In [9]:
#Using pandas, going to create the dataframe list:
df_list=[]
for d in data_list:
    temp_df=pd.read_csv(d+".csv")
    df_list.append(temp_df)

In [10]:
#The last column is empty, must be removed
df_list[0].head(5)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,36846.848285,37343.912963,37583.840323,38865.188195,41679.238190,42501.641554,34971.009913,42698.359872,NaN,NaN
1,Africa Eastern and Southern,AFE,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3470.550675,3498.125396,3591.099809,3635.564622,3724.868177,3777.972368,3621.058106,3839.470058,NaN,NaN
2,Afghanistan,AFG,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2110.829568,2128.125938,2023.834656,2096.093111,2109.929296,2167.704111,2076.138380,1665.805842,NaN,NaN
3,Africa Western and Central,AFW,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,4143.177054,4075.994459,3998.951876,4045.303263,4160.540965,4264.731035,4174.504565,4409.450961,NaN,NaN
4,Angola,AGO,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,8123.048065,7274.090475,7027.146634,7216.061373,7042.923829,6881.076241,6362.636076,6491.125578,NaN,NaN


In [11]:
#Running the deletion
for n in range(len(df_list)):
    df_list[n]=df_list[n][df_list[n].columns[:-1]]

In [12]:
#Checking
df_list[0].head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,36932.184164,36846.848285,37343.912963,37583.840323,38865.188195,41679.238190,42501.641554,34971.009913,42698.359872,NaN
1,Africa Eastern and Southern,AFE,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3333.520911,3470.550675,3498.125396,3591.099809,3635.564622,3724.868177,3777.972368,3621.058106,3839.470058,NaN
2,Afghanistan,AFG,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2062.059176,2110.829568,2128.125938,2023.834656,2096.093111,2109.929296,2167.704111,2076.138380,1665.805842,NaN
3,Africa Western and Central,AFW,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3950.948889,4143.177054,4075.994459,3998.951876,4045.303263,4160.540965,4264.731035,4174.504565,4409.450961,NaN
4,Angola,AGO,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,7643.921944,8123.048065,7274.090475,7027.146634,7216.061373,7042.923829,6881.076241,6362.636076,6491.125578,NaN
